In [28]:
import string 
import re 
from os import listdir 
from collections import Counter 
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer 
from keras.utils.vis_utils import plot_model 
from keras.models import Sequential 
from keras.layers import Dense
from keras.layers import Flatten 
from pandas import DataFrame 
from matplotlib import pyplot
from keras.layers import Embedding 
from keras.models import Model 
from keras.layers import Input  
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D 
from keras.layers.convolutional import MaxPooling1D
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from pickle import dump
from pickle import load 
from keras.layers.merge import concatenate




In [2]:
# load doc into memory 
def load_doc(filename): 
    # open the file as read only 
    file = open(filename,'r') 
    # read all text 
    text = file.read() 
    # close the file 
    file.close() 
    return text

In [6]:
# turn a doc into clean tokens 
def clean_doc(doc): 
    # split into tokens by white space 
    tokens = doc.split() 
    # prepare regex for char filtering 
    re_punc = re.compile('[%s]' % re.escape(string.punctuation)) 
    # remove punctuation from each word 
    tokens = [re_punc.sub('', w) for w in tokens] 
    # remove remaining tokens that are not alphabetic 
    tokens = [word for word in tokens if word.isalpha()] 
    # filter out stop words
    stop_words = set(stopwords.words('english')) 
    tokens = [w for w in tokens if not w in stop_words] 
    # filter out short tokens 
    tokens = [word for word in tokens if len(word) > 1] 
    tokens = ' '.join(tokens) 
    return tokens

In [11]:
# load all docs in a directory 
def process_docs(directory, is_train): 
    documents = list()
    # walk through all files in the folder 
    for filename in listdir(directory): 
        # skip any reviews in the test set 
        if is_train and filename.startswith('cv9'): 
            continue 
        if not is_train and not filename.startswith('cv9'): 
            continue     
        # create the full path of the file to open 
        path = directory + '/' + filename 
        # load the doc 
        doc = load_doc(path) 
        # clean doc 
        tokens = clean_doc(doc) 
        documents.append(tokens)
    return documents

In [12]:
# load and clean a dataset 
def load_clean_dataset(is_train): 
    # load documents 
    neg = process_docs('txt_sentoken/neg', is_train) 
    pos = process_docs('txt_sentoken/pos', is_train) 
    docs = neg + pos 
    # prepare labels 
    labels = [0 for _ in range(len(neg))] + [1 for _ in range(len(pos))] 
    return docs, labels

In [13]:
# save a dataset to file 
def save_dataset(dataset, filename): 
    dump(dataset, open(filename, 'wb')) 
    print('Saved: %s' % filename)


In [14]:
# load and clean all reviews 
train_docs, ytrain = load_clean_dataset(True) 
test_docs, ytest = load_clean_dataset(False) 
# save training datasets 
save_dataset([train_docs, ytrain], 'train.pkl') 
save_dataset([test_docs, ytest], 'test.pkl')



Saved: train.pkl
Saved: test.pkl


In [15]:
# load a clean dataset 
def load_dataset(filename): 
    return load(open(filename, 'rb'))


In [16]:
# fit a tokenizer 
def create_tokenizer(lines): 
    tokenizer = Tokenizer() 
    tokenizer.fit_on_texts(lines) 
    return tokenizer

In [17]:
# calculate the maximum document length 
def max_length(lines): 
    return max([len(s.split()) for s in lines])


In [24]:
# integer encode and pad documents 
def encode_text(tokenizer, lines,length): 
    # integer encode 
    encoded = tokenizer.texts_to_sequences(lines) 
    # pad sequences 
    padded = pad_sequences(encoded, maxlen=length, padding='post') 
    return padded

In [30]:
# define the model 
def define_model(length, vocab_size): 
    # channel 1 
    inputs1 = Input(shape=(length,)) 
    embedding1 = Embedding(vocab_size, 100)(inputs1) 
    conv1 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding1) 
    drop1 = Dropout(0.5)(conv1) 
    pool1 = MaxPooling1D(pool_size=2)(drop1) 
    flat1 = Flatten()(pool1) 
    # channel 2 
    inputs2 = Input(shape=(length,)) 
    embedding2 = Embedding(vocab_size, 100)(inputs2) 
    conv2 = Conv1D(filters=32, kernel_size=6, activation='relu')(embedding2) 
    drop2 = Dropout(0.5)(conv2) 
    pool2 = MaxPooling1D(pool_size=2)(drop2) 
    flat2 = Flatten()(pool2)
    # channel 3 
    inputs3 = Input(shape=(length,)) 
    embedding3 = Embedding(vocab_size, 100)(inputs3) 
    conv3 = Conv1D(filters=32, kernel_size=8, activation='relu')(embedding3) 
    drop3 = Dropout(0.5)(conv3) 
    pool3 = MaxPooling1D(pool_size=2)(drop3) 
    flat3 = Flatten()(pool3) 
    # merge 
    merged = concatenate([flat1, flat2, flat3]) 
    # interpretation 
    dense1 = Dense(10, activation='relu')(merged) 
    outputs = Dense(1, activation='sigmoid')(dense1) 
    model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs) 
    # compile 
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
    # summarize 
    model.summary() 
    plot_model(model, show_shapes=True, to_file='model_multichannel.png') 
    return model


In [31]:
# load training dataset 
trainLines, trainLabels = load_dataset('train.pkl') 
# create tokenizer 
tokenizer = create_tokenizer(trainLines) 
# calculate max document length 
length = max_length(trainLines) 
print('Max document length: %d' % length) 
# calculate vocabulary size 
vocab_size = len(tokenizer.word_index) + 1 
print('Vocabulary size: %d' % vocab_size) 
# encode data 
trainX = encode_text(tokenizer, trainLines, length) 
# define model 
model = define_model(length, vocab_size) 
# fit model 
model.fit([trainX,trainX,trainX], trainLabels, epochs=7, batch_size=16) 
# save the model 
model.save('model_multichannel.h5')


Max document length: 1380
Vocabulary size: 44277
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 1380)         0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, 1380)         0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            (None, 1380)         0                                            
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, 1380, 100)    4427700     input_7[0][0]                    
____________________________________________________________

In [34]:
testLines, testLabels = load_dataset('test.pkl') 
testX = encode_text(tokenizer, testLines, length) 
# load the model 
model = load_model('model_multichannel.h5') 
# evaluate model on training dataset 
_, acc = model.evaluate([trainX,trainX,trainX], trainLabels, verbose=0) 
print('Train Accuracy: %.2f' % (acc*100)) 
# evaluate model on test dataset dataset 
_, acc = model.evaluate([testX,testX,testX], testLabels, verbose=0) 
print('Test Accuracy: %.2f' % (acc*100))


Train Accuracy: 100.00
Test Accuracy: 84.50
